In [1]:
# FIX THIS NOTEBOOK

# Automatically re-import files when updated
%load_ext autoreload
%autoreload 2  
import time, json, pandas as pd, IPython
# Load packages
from models import *
from utils import *
from matplotlib import pyplot as plt

# Load models

In [2]:
# Load models
with tf.keras.utils.custom_object_scope({
    'Inception': Inception,
    "downsampler": downsampler,
    "upsampler": upsampler
    }):
    decoder = tf.keras.models.load_model("decoder.keras")
    encoder = tf.keras.models.load_model("encoder.keras")
    classifier = tf.keras.models.load_model("classifier.keras")

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


# Visualize results

In [ ]:
# Get data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
# Convert to tf dataset 
data_train = color_dataset_from_arrays(X_train, y_train)
data_test = color_dataset_from_arrays(X_test, y_test)

# Split
imgs, labels = list(data_test.take(1))[0]
# For originals
original_predictions = classifier(imgs)
# For adversarials
adv_imgs = fgsm(classifier, tf.keras.losses.CategoricalCrossentropy(), imgs, labels, tf.constant(0.01))
adv_predictions = classifier(adv_imgs)
# For reconstructions
z = encoder(adv_imgs)
reconstructed = decoder(z)
predictions = classifier(reconstructed)

# Plot
for (img, orig_pred, adv_img, rec, label, pred, adv_pred) in zip(imgs, original_predictions, adv_imgs, reconstructed, labels, predictions, adv_predictions):
    # Clear display
    IPython.display.clear_output()
    # Create subplots
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
    # Plot the original image
    ax1.imshow(img)
    ax1.set_title("Original image")
    ax1.axis("off")
    # Plot the original image
    ax2.imshow(adv_img)
    ax2.set_title("Adversarial example")
    ax2.axis("off")
    # Plot the reconstructed image
    ax3.imshow(rec)
    ax3.set_title("Reconstructed image")
    ax3.axis("off")
    # Add context
    print(f"True label: {tf.math.argmax(label).numpy()}\
        \npredicted label on original image: {tf.math.argmax(orig_pred).numpy()}\
        \nPredicted label on adversarial example: {tf.math.argmax(adv_pred).numpy()}\
        \nPredicted label on reconstructed image: {tf.math.argmax(pred).numpy()}")
    # Show for some time
    plt.show()
    time.sleep(2)

# Training progress

In [ ]:
# Plot training progress
history = json.load(open("history.json", mode="r"))
history = pd.DataFrame.from_dict(history, orient="index")
history["classifier_loss"].plot();
history["ae_loss"].plot(secondary_y=True);
plt.legend();

# Plot models

In [5]:
# Plot models (inception done manually)
tf.keras.utils.plot_model(encoder, show_layer_names=False, to_file="encoder.png");
block = downsampler(1, 6, 1, 1)
block.build((BATCH_SIZE, None, None, 3))
tf.keras.utils.plot_model(block, show_layer_names=False, to_file="downsampler.png");